# Introduction

The webgpu package utilizes two versions of python: One running some "real" Python for computations. And a second instance of Python using pyodide within the browser.

If you run notebooks the "real" Python is the one running in the ipykernel, which is most likely a Python process on your PC, but can also be a jupyterlite instance or a Python running on some server.

Importing webgpu.jupyter starts this additional python instance in the browser. You can open the developer console (usually F12 and then to "Console" in the top menu) to see the print outputs from pyodide.

In [ ]:
import webgpu.jupyter as wj

## Pyodide

When we want to execute code on the Pyodide side we have magic for this (you see the output in the developer console). This cell might need a few seconds to execute until the pyodide kernel is started up.

In [ ]:
%%pyodide
print("Hi from Pyodide!")

Sometimes we want to execute code in Pyodide and in the kernel, for this we also provide some magic:

In [ ]:
%%pyodide_and_kernel
print("Hi from both!")

We can send objects to pyodide (deep copy via pickle):

In [ ]:
wj.pyodide.my_list = [1, 2, 3, 4]

In [ ]:
%%pyodide
print(my_list)

Not all packages are default available in pyodide:

In [ ]:
%%pyodide
import scipy

Packages from the pyodide repository (https://pyodide.org/en/stable/usage/packages-in-pyodide.html) or pure python packages from pip can be installed using micropip:

In [ ]:
%%pyodide
# pyodide wasm compiled scipy
import micropip
await micropip.install("scipy")
import scipy
print(scipy)
# pure python package from pip
await micropip.install("emoji")
import emoji
print(emoji.emojize(":thumbs_up:"))

## Render Objects

And create renderer objects in the kernel and Draw them with webgpu.jupyter.Draw command. The Draw command creates a jupyter canvas, pickles the given objects and sends them and the canvas_id to the pyodide python which has access to the gpu to draw using our webgpu bindings.

In [ ]:
from webgpu.triangles import TriangulationRenderer

points = [(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 0, 0), (1, 1, 1), (0, 1, 0)]
trigs = TriangulationRenderer(points, color=(1, 1, 0, 1))
wj.Draw(trigs)

You can draw multiple objects into one scene:

In [ ]:
from webgpu.vectors import VectorRenderer

p = points[:3]
v = points[3:]
vrender = VectorRenderer(points=p, vectors=v, size=0.2)
wj.Draw([trigs, vrender])

## Controls

Adding options to the control sections of a scene can be done by the `scene.gui` object.

Note that the function that is given must be a standalone function. `RenderObjects` can be passed as objects (either one or a list of RenderObjects). Since we must change the pyodide side version of the object. Internally we handle references to `RenderObjects` via their unique `object._id`. The passed function is then added as a callback to the slider control on the pyodide side. Getting the pyodide version of the render object as an input.
After each callback a scene.redraw is called automatically.

In [ ]:
scene = wj.Draw([trigs, vrender])


def change_size(vr, size):
    vr.vec_uniforms.size = size
    vr.vec_uniforms.update_buffer()


scene.gui.slider(
    value=0.2, func=change_size, objects=vrender, min=0.0, max=1.0, label="Size"
)